# Date encoding

In [8]:
import imp
import numpy as np
import pandas as pd
import xgboost as xgb
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.base import BaseEstimator
import xgboost as xgb
from lightgbm import LGBMRegressor
plt.rcParams['figure.figsize'] = [20, 20]

In [37]:
problem = imp.load_source('', 'problem.py')
X_df, y_array = problem.get_train_data()
X_df.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
0,2012-06-19,ORD,DFW,12.875000,9.812647
1,2012-09-10,LAS,DEN,14.285714,9.466734
2,2012-10-05,DEN,LAX,10.863636,9.035883
3,2011-10-09,ATL,ORD,11.480000,7.990202
4,2012-02-21,DEN,SFO,11.450000,9.517159


In [38]:
ext_data = pd.read_csv('submissions/starting_kit/external_data.csv')
ext_data.head()

,Departure,Arrival,Distance,DateOfDeparture,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Number_hab,Revenue,Oil_price,dep_encod,ar_encod
0,LAS,ORD,2434,2011-09-01,34,28,22,23,22,19,84,70,56,1015,1013,1011,16,15,11,26,9,37.0,0,1,NaN,197,2712920,52273,88.93,10.79,11.26
1,LAS,ORD,2434,2011-09-02,33,29,24,22,21,19,79,64,49,1013,1012,1011,16,15,11,27,12,37.0,0,4,NaN,215,2712920,52273,86.57,10.79,11.26
2,LAS,ORD,2434,2011-09-03,32,26,21,22,20,18,93,70,46,1013,1012,1011,16,15,8,32,10,39.0,0.51,6,Rain-Thunderstorm,270,2712920,52273,93.79,10.79,11.26
3,LAS,ORD,2434,2011-09-04,23,19,14,19,13,9,93,70,47,1017,1013,1010,16,16,16,35,19,45.0,0,5,NaN,312,2712920,52273,93.79,10.79,11.26
4,LAS,ORD,2434,2011-09-05,18,15,11,10,7,6,77,64,51,1021,1020,1017,16,16,16,42,24,56.0,0,3,NaN,5,2712920,52273,83.52,10.79,11.26


In [48]:
X_encoded = X_df
external_data = ext_data[['DateOfDeparture', 'Departure', 'Arrival', 'Distance',
                                'dep_encod', 'ar_encod',
                                'Number_hab'
                         ]]
X_encoded = pd.merge(
            X_encoded, external_data, how='left',
            left_on=['DateOfDeparture', 'Arrival', 'Departure'],
            right_on=['DateOfDeparture', 'Arrival', 'Departure'],
            sort=False)

X_encoded.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,Distance,dep_encod,ar_encod,Number_hab
0,2012-06-19,ORD,DFW,12.875000,9.812647,1292,11.29,11.09,1241700
1,2012-09-10,LAS,DEN,14.285714,9.466734,1010,10.79,10.68,634542
2,2012-10-05,DEN,LAX,10.863636,9.035883,1387,10.68,11.43,3852782
3,2011-10-09,ATL,ORD,11.480000,7.990202,977,10.99,11.26,2712920
4,2012-02-21,DEN,SFO,11.450000,9.517159,1554,10.68,11.23,827420


In [49]:
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure']))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))
X_encoded = X_encoded.drop('Departure', axis=1)
X_encoded = X_encoded.drop('Arrival', axis=1)

X_encoded.loc[:, 'ATL': 'SFO'] = X_encoded.loc[:, 'ATL': 'SFO'].mul(X_encoded.loc[:, 'dep_encod'], axis='rows')
arr = X_encoded.loc[:, 'a_ATL': 'a_SFO'].mul(- 1 * X_encoded.loc[:, 'ar_encod'], axis='rows')

arr.columns = X_encoded.loc[:, 'ATL': 'SFO'].columns
X_encoded.loc[:, 'ATL': 'SFO'] = X_encoded.loc[:, 'ATL': 'SFO'].add(arr, axis=1)

X_encoded.drop(['dep_encod', 'ar_encod'], axis=1, inplace=True)
X_encoded.drop(X_encoded.loc[:, 'a_ATL': 'a_SFO'].columns, axis=1, inplace=True)
X_encoded.head()

,DateOfDeparture,WeeksToDeparture,std_wtd,Distance,Number_hab,ATL,BOS,CLT,DEN,DFW,DTW,EWR,IAH,JFK,LAS,LAX,LGA,MCO,MIA,MSP,ORD,PHL,PHX,SEA,SFO
0,2012-06-19,12.875000,9.812647,1292,1241700,0.00,0.0,0.0,0.00,-11.09,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,11.29,0.0,0.0,0.0,0.00
1,2012-09-10,14.285714,9.466734,1010,634542,0.00,0.0,0.0,-10.68,0.00,0.0,0.0,0.0,0.0,10.79,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
2,2012-10-05,10.863636,9.035883,1387,3852782,0.00,0.0,0.0,10.68,0.00,0.0,0.0,0.0,0.0,0.00,-11.43,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
3,2011-10-09,11.480000,7.990202,977,2712920,10.99,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,-11.26,0.0,0.0,0.0,0.00
4,2012-02-21,11.450000,9.517159,1554,827420,0.00,0.0,0.0,10.68,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,-11.23


In [50]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [51]:
X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
X_encoded['day'] = X_encoded['DateOfDeparture'].dt.day
X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("2011-09-01")).days)
X_encoded.head()

,DateOfDeparture,WeeksToDeparture,std_wtd,Distance,Number_hab,ATL,BOS,CLT,DEN,DFW,DTW,EWR,IAH,JFK,LAS,LAX,LGA,MCO,MIA,MSP,ORD,PHL,PHX,SEA,SFO,year,month,day,weekday,week,n_days
0,2012-06-19,12.875000,9.812647,1292,1241700,0.00,0.0,0.0,0.00,-11.09,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,11.29,0.0,0.0,0.0,0.00,2012,6,19,1,25,292
1,2012-09-10,14.285714,9.466734,1010,634542,0.00,0.0,0.0,-10.68,0.00,0.0,0.0,0.0,0.0,10.79,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,2012,9,10,0,37,375
2,2012-10-05,10.863636,9.035883,1387,3852782,0.00,0.0,0.0,10.68,0.00,0.0,0.0,0.0,0.0,0.00,-11.43,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,2012,10,5,4,40,400
3,2011-10-09,11.480000,7.990202,977,2712920,10.99,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,-11.26,0.0,0.0,0.0,0.00,2011,10,9,6,40,38
4,2012-02-21,11.450000,9.517159,1554,827420,0.00,0.0,0.0,10.68,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,-11.23,2012,2,21,1,8,173


In [52]:
X_encoded = encode(X_encoded, 'month', 12)
X_encoded = encode(X_encoded, 'week', 52)
X_encoded = encode(X_encoded, 'day', 365)
X_encoded = encode(X_encoded, 'weekday', 6)

In [53]:
X_encoded = X_encoded.drop(['DateOfDeparture', 'year', 'month', 'day', 'weekday', 'week', 'std_wtd'], axis=1)
X_encoded.head()

,WeeksToDeparture,Distance,Number_hab,ATL,BOS,CLT,DEN,DFW,DTW,EWR,IAH,JFK,LAS,LAX,LGA,MCO,MIA,MSP,ORD,PHL,PHX,SEA,SFO,n_days,month_sin,month_cos,week_sin,week_cos,day_sin,day_cos,weekday_sin,weekday_cos
0,12.875000,1292,1241700,0.00,0.0,0.0,0.00,-11.09,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,11.29,0.0,0.0,0.0,0.00,292,1.224647e-16,-1.000000e+00,0.120537,-0.992709,0.321270,0.946988,8.660254e-01,0.5
1,14.285714,1010,634542,0.00,0.0,0.0,-10.68,0.00,0.0,0.0,0.0,0.0,10.79,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,375,-1.000000e+00,-1.836970e-16,-0.970942,-0.239316,0.171293,0.985220,0.000000e+00,1.0
2,10.863636,1387,3852782,0.00,0.0,0.0,10.68,0.00,0.0,0.0,0.0,0.0,0.00,-11.43,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,400,-8.660254e-01,5.000000e-01,-0.992709,0.120537,0.085965,0.996298,-8.660254e-01,-0.5
3,11.480000,977,2712920,10.99,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,-11.26,0.0,0.0,0.0,0.00,38,-8.660254e-01,5.000000e-01,-0.992709,0.120537,0.154309,0.988023,-2.449294e-16,1.0
4,11.450000,1554,827420,0.00,0.0,0.0,10.68,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,-11.23,173,8.660254e-01,5.000000e-01,0.822984,0.568065,0.353676,0.935368,8.660254e-01,0.5


from sklearn.decomposition import PCA 
from sklearn import preprocessing

reserv = preprocessing.scale(X_encoded.loc[:, 'WeeksToDeparture' : 'std_wtd'])
meteo = preprocessing.scale(X_encoded.loc[:, 'Mean TemperatureC' : 'Max Wind SpeedKm/h'])
pca = PCA(n_components=1)
reserv = pca.fit_transform(reserv)

pca = PCA(n_components=2)
meteo = pca.fit_transform(meteo)

meteo = pd.DataFrame(data = meteo, columns = ['Princ. Comp. ' + str(i) for i in range(2)])

X_encoded = pd.concat([X_encoded, meteo], axis = 1)
X_encoded['reserv'] = reserv
X_encoded.head()

X_encoded['Events'].fillna(0, inplace=True)
X_encoded['Events'] = X_encoded['Events'].replace(['Rain','Fog'], 1)
X_encoded['Events'] = X_encoded['Events'].replace(['Rain-Thunderstorm','Fog-Rain-Thunderstorm', \
                                                'Rain-Snow','Snow','Fog-Rain','Thunderstorm','Fog-Snow', \
                                                'Fog-Rain-Snow','Fog-Rain-Snow-Thunderstorm', \
                                                'Rain-Snow-Thunderstorm','Rain-Hail-Thunderstorm', \
                                                'Fog-Rain-Hail-Thunderstorm', \
                                                'Rain-Thunderstorm-Tornado'], 2)

X_encoded.drop(['WeeksToDeparture', 'std_wtd', 'Mean TemperatureC',
                'MeanDew PointC', 'Mean Humidity', 'Min VisibilitykM',
                'Max Wind SpeedKm/h',], axis=1, inplace=True)
X_encoded.head()

X_encoded['Weeks_to_dep_int'] = pd.qcut(X_encoded['WeeksToDeparture'], 4)
X_encoded.drop(['Weeks_to_dep_int'], axis=1, inplace=True)

In [54]:
X_encoded.loc[X_encoded['WeeksToDeparture'] <= 9.524, 'WeeksToDeparture'] = 0
X_encoded.loc[(X_encoded['WeeksToDeparture'] > 9.524) & (X_encoded['WeeksToDeparture'] <= 11.3), 'WeeksToDeparture'] = 1
X_encoded.loc[(X_encoded['WeeksToDeparture'] > 11.3) & (X_encoded['WeeksToDeparture'] <= 13.24), 'WeeksToDeparture'] = 2
X_encoded.loc[ X_encoded['WeeksToDeparture'] > 13.24, 'WeeksToDeparture'] = 3
X_encoded['WeeksToDeparture'] = X_encoded['WeeksToDeparture'].astype(int)

In [55]:
X_encoded.head()

,WeeksToDeparture,Distance,Number_hab,ATL,BOS,CLT,DEN,DFW,DTW,EWR,IAH,JFK,LAS,LAX,LGA,MCO,MIA,MSP,ORD,PHL,PHX,SEA,SFO,n_days,month_sin,month_cos,week_sin,week_cos,day_sin,day_cos,weekday_sin,weekday_cos
0,2,1292,1241700,0.00,0.0,0.0,0.00,-11.09,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,11.29,0.0,0.0,0.0,0.00,292,1.224647e-16,-1.000000e+00,0.120537,-0.992709,0.321270,0.946988,8.660254e-01,0.5
1,3,1010,634542,0.00,0.0,0.0,-10.68,0.00,0.0,0.0,0.0,0.0,10.79,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,375,-1.000000e+00,-1.836970e-16,-0.970942,-0.239316,0.171293,0.985220,0.000000e+00,1.0
2,1,1387,3852782,0.00,0.0,0.0,10.68,0.00,0.0,0.0,0.0,0.0,0.00,-11.43,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,400,-8.660254e-01,5.000000e-01,-0.992709,0.120537,0.085965,0.996298,-8.660254e-01,-0.5
3,2,977,2712920,10.99,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,-11.26,0.0,0.0,0.0,0.00,38,-8.660254e-01,5.000000e-01,-0.992709,0.120537,0.154309,0.988023,-2.449294e-16,1.0
4,2,1554,827420,0.00,0.0,0.0,10.68,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,-11.23,173,8.660254e-01,5.000000e-01,0.822984,0.568065,0.353676,0.935368,8.660254e-01,0.5


In [56]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_array, test_size=0.2, random_state=0)

xgb1 = xgb.XGBRegressor()

In [65]:
data_dmatrix = xgb.DMatrix(data=X_encoded,label=y_array)
parameters = {'nthread':1,
              'objective':'reg:squarederror',
              'learning_rate': .16, 
              'max_depth': 4,
              'min_child_weight': 4,
              'n_estimators': 1000,
              'reg_lambda':0.001,
              'gamma':0}

cv_results = xgb.cv(dtrain=data_dmatrix, params=parameters, nfold=3,
                    num_boost_round=300,early_stopping_rounds=5,metrics="rmse", as_pandas=True, seed=123)

In [66]:
print((cv_results["train-rmse-mean"]).tail(1))
print((cv_results["test-rmse-mean"]).tail(1))

225    0.278873
Name: train-rmse-mean, dtype: float64
225    0.385782
Name: test-rmse-mean, dtype: float64


In [ ]:
xg_reg = xgb.train(params=parameters, dtrain=data_dmatrix, num_boost_round=10)
xgb.plot_importance(xg_reg, max_num_features=40)
plt.rcParams['figure.figsize'] = [20, 20]
plt.show()

In [ ]:
gbm = LGBMRegressor(num_leaves=31,
                    learning_rate=0.1,
                    n_estimators=400)

In [ ]:
from sklearn.metrics import mean_squared_error
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='rmse',
        early_stopping_rounds=5)
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)
